In [1]:
"""from google.colab import drive
drive.mount('/content/drive')"""

"from google.colab import drive\ndrive.mount('/content/drive')"

In [2]:
import os
import xml.etree.ElementTree as ET

## 1 Images and labels - ne pokretati opet

In [3]:
"""def read_content(xml_file: str):

    tree = ET.parse(xml_file)
    root = tree.getroot()

    list_with_all_boxes = []
    filename = root.find("filename").text

    size_node = root.find("size")
    h = int(size_node.find("height").text)
    w = int(size_node.find("width").text)

    for w_idx, boxes in enumerate(root.iter("object")):
        ymin, xmin, ymax, xmax = None, None, None, None

        ymin = int(boxes.find("bndbox/ymin").text)
        xmin = int(boxes.find("bndbox/xmin").text)
        ymax = int(boxes.find("bndbox/ymax").text)
        xmax = int(boxes.find("bndbox/xmax").text)
        name = str(boxes.find("name").text).upper()

        row = [filename, w_idx, w, h, xmin, ymin, xmax, ymax, name]
        list_with_all_boxes.append(row)

    return list_with_all_boxes"""

'def read_content(xml_file: str):\n\n    tree = ET.parse(xml_file)\n    root = tree.getroot()\n\n    list_with_all_boxes = []\n    filename = root.find("filename").text\n\n    size_node = root.find("size")\n    h = int(size_node.find("height").text)\n    w = int(size_node.find("width").text)\n\n    for w_idx, boxes in enumerate(root.iter("object")):\n        ymin, xmin, ymax, xmax = None, None, None, None\n\n        ymin = int(boxes.find("bndbox/ymin").text)\n        xmin = int(boxes.find("bndbox/xmin").text)\n        ymax = int(boxes.find("bndbox/ymax").text)\n        xmax = int(boxes.find("bndbox/xmax").text)\n        name = str(boxes.find("name").text).upper()\n\n        row = [filename, w_idx, w, h, xmin, ymin, xmax, ymax, name]\n        list_with_all_boxes.append(row)\n\n    return list_with_all_boxes'

In [4]:
def generate_word_labels(path, labels):
    """
    Format
    filename, img_width, img_height, word_idx, xmin, ymin, xmax, ymax, word
    """
    """with open(f"{path}/word_labels.txt", "w", encoding="utf8") as file:
        for label in labels:
            xml_file = f"{path}/{label}"
            boxes = read_content(xml_file)
            label = label.removesuffix(".xml")
            for box in boxes:
                b = [str(i) for i in box]
                file.write(f'{"|".join(b)}\n')"""

In [5]:
"""path = "/content/drive/MyDrive/VI_projekat_proba/dataset/ground_truth"
labels = os.listdir(path)
labels = [l for l in labels if l.endswith(".xml")]

generate_word_labels(path, labels)"""

'path = "/content/drive/MyDrive/VI_projekat_proba/dataset/ground_truth"\nlabels = os.listdir(path)\nlabels = [l for l in labels if l.endswith(".xml")]\n\ngenerate_word_labels(path, labels)'

In [6]:
"""def extract_rectangle(image, top_left, bottom_right, wiggle_room=None):
    if wiggle_room is not None:
        w, h, _ = image.shape
        x1p, y1p = top_left
        x2p, y2p = bottom_right

        top_left = [
            x1p - wiggle_room if 0 <= x1p - wiggle_room <= w else x1p,
            y1p - wiggle_room if 0 <= y1p - wiggle_room <= h else y1p,
        ]
        bottom_right = [
            x2p + wiggle_room if 0 <= x2p + wiggle_room <= w else x2p,
            y2p + wiggle_room if 0 <= y2p + wiggle_room <= h else y2p,
        ]
    x1, y1 = top_left
    x2, y2 = bottom_right
    extracted_region = image[y1:y2, x1:x2]
    return extracted_region"""

'def extract_rectangle(image, top_left, bottom_right, wiggle_room=None):\n    if wiggle_room is not None:\n        w, h, _ = image.shape\n        x1p, y1p = top_left\n        x2p, y2p = bottom_right\n\n        top_left = [\n            x1p - wiggle_room if 0 <= x1p - wiggle_room <= w else x1p,\n            y1p - wiggle_room if 0 <= y1p - wiggle_room <= h else y1p,\n        ]\n        bottom_right = [\n            x2p + wiggle_room if 0 <= x2p + wiggle_room <= w else x2p,\n            y2p + wiggle_room if 0 <= y2p + wiggle_room <= h else y2p,\n        ]\n    x1, y1 = top_left\n    x2, y2 = bottom_right\n    extracted_region = image[y1:y2, x1:x2]\n    return extracted_region'

In [7]:
"""import cv2

word_labels = "/content/drive/MyDrive/VI_projekat_proba/dataset/ground_truth/word_labels.txt"

path = "/content/drive/MyDrive/VI_projekat_proba/dataset/images"

labels = os.listdir(path)
labels = [l for l in labels if l.endswith(".xml")]

def generate_word_images(word_labels, src, dest):
    with open(word_labels, "r", encoding="utf8") as read:
        with open(f"{dest}/word.txt", "w", encoding="utf8") as write:
            while line := read.readline():
                line = line.removesuffix("\n").split("|")
                filename, word_idx, w, h, xmin, ymin, xmax, ymax, word = line
                img = cv2.imread(f"{src}/{filename}")
                if img is None:
                    print(f"{src}/{filename}")
                word_img = extract_rectangle(img, [int(xmin), int(ymin)], [int(xmax), int(ymax)])
                filename = filename.removesuffix(".jpg")
                img_path = f"{dest}/word/{filename}_{word_idx}.jpg"
                try:
                    cv2.imwrite(img_path, word_img)
                    line[0] = f"{filename}_{word_idx}.jpg"
                    write.write(f"{'|'.join(line)}\n")
                except Exception as e:
                    print(e)

labels_w_path = "/content/drive/MyDrive/VI_projekat_proba/dataset/labels_w"
if not os.path.isdir(labels_w_path):
    os.mkdir(labels_w_path)

labels_w_word_path = "/content/drive/MyDrive/VI_projekat_proba/dataset/labels_w/word"
if not os.path.isdir(labels_w_word_path):
    os.mkdir(labels_w_word_path)

generate_word_images(word_labels, path, labels_w_path)"""

'import cv2\n\nword_labels = "/content/drive/MyDrive/VI_projekat_proba/dataset/ground_truth/word_labels.txt"\n\npath = "/content/drive/MyDrive/VI_projekat_proba/dataset/images"\n\nlabels = os.listdir(path)\nlabels = [l for l in labels if l.endswith(".xml")]\n\ndef generate_word_images(word_labels, src, dest):\n    with open(word_labels, "r", encoding="utf8") as read:\n        with open(f"{dest}/word.txt", "w", encoding="utf8") as write:\n            while line := read.readline():\n                line = line.removesuffix("\n").split("|")\n                filename, word_idx, w, h, xmin, ymin, xmax, ymax, word = line\n                img = cv2.imread(f"{src}/{filename}")\n                if img is None:\n                    print(f"{src}/{filename}")\n                word_img = extract_rectangle(img, [int(xmin), int(ymin)], [int(xmax), int(ymax)])\n                filename = filename.removesuffix(".jpg")\n                img_path = f"{dest}/word/{filename}_{word_idx}.jpg"\n              

In [8]:
"""from sklearn.model_selection import train_test_split

def split_train_set(src_txt: str, out: str, train_size=0.7, val_size=0.2, test_size=0.1, random_state=None):
    assert abs(train_size+val_size+test_size-1.0) < 1e-5, "Train, validation, and test sizes must sum to 1.0"
    elements = []
    with open(src_txt, "r", encoding="utf8") as read:
        while line := read.readline():
            elements.append(line)

    if not elements:
        return 0,0,0

    train_val_data, test_data = train_test_split(elements, test_size=test_size, random_state=random_state)
    val_relative_size = val_size / (train_size + val_size)
    train_data, val_data = train_test_split(train_val_data, test_size=val_relative_size, random_state=random_state)
    with open(f"{out}/trainset.txt", "w", encoding="utf8") as train:
        for el in train_data:
            train.write(el)

    with open(f"{out}/testset.txt", "w", encoding="utf8") as test:
        for el in test_data:
            test.write(el)

    with open(f"{out}/validset.txt", "w", encoding="utf8") as val:
        for el in val_data:
            val.write(el)

    return len(train_data), len(val_data), len(test_data)

_out = "/content/drive/MyDrive/VI_projekat_proba/dataset/labels_w"
_in = "/content/drive/MyDrive/VI_projekat_proba/dataset/labels_w/word.txt"
if not os.path.isdir(_out):
    os.mkdir(_out)

split_train_set(_in, _out)"""

'from sklearn.model_selection import train_test_split\n\ndef split_train_set(src_txt: str, out: str, train_size=0.7, val_size=0.2, test_size=0.1, random_state=None):\n    assert abs(train_size+val_size+test_size-1.0) < 1e-5, "Train, validation, and test sizes must sum to 1.0"\n    elements = []\n    with open(src_txt, "r", encoding="utf8") as read:\n        while line := read.readline():\n            elements.append(line)\n\n    if not elements:\n        return 0,0,0\n\n    train_val_data, test_data = train_test_split(elements, test_size=test_size, random_state=random_state)\n    val_relative_size = val_size / (train_size + val_size)\n    train_data, val_data = train_test_split(train_val_data, test_size=val_relative_size, random_state=random_state)\n    with open(f"{out}/trainset.txt", "w", encoding="utf8") as train:\n        for el in train_data:\n            train.write(el)\n\n    with open(f"{out}/testset.txt", "w", encoding="utf8") as test:\n        for el in test_data:\n          

## 2 Google Drive Environment

### 2.1 TensorFlow 2.x

In [9]:
#%tensorflow_version 2.x
import tensorflow as tf

device_name = tf.test.gpu_device_name()

if device_name != "/device:GPU:0":
    raise SystemError("GPU device not found")

print("Found GPU at: {}".format(device_name))

2025-06-16 14:45:52.372655: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750077952.392218   17001 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750077952.399130   17001 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1750077952.415696   17001 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750077952.415733   17001 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1750077952.415735   17001 computation_placer.cc:177] computation placer alr

Found GPU at: /device:GPU:0


I0000 00:00:1750077955.093999   17001 gpu_device.cc:2019] Created device /device:GPU:0 with 9711 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:05:00.0, compute capability: 8.6


### 2.2 Google Drive

In [10]:
"""%cd "/content/drive/MyDrive/VI_projekat_proba/dataset/src/"
!ls -l"""

'%cd "/content/drive/MyDrive/VI_projekat_proba/dataset/src/"\n!ls -l'

## 3 Set Python Classes

### 3.1 Environment

In [11]:
"""import datetime
import string

# define parameters
source = "labels_w"
arch = "flor"
epochs = 1000
batch_size = 16

# define paths
source_path = os.path.join("..", "data", f"{source}.hdf5")
output_path = os.path.join("..", "output", source, arch)
target_path = os.path.join(output_path, "checkpoint_weights.weights.h5")
os.makedirs(output_path, exist_ok=True)

# define input size, number max of chars per line and list of valid chars
input_size = (1024, 128, 1)
max_text_length = 128
charset_base = string.printable[:95]
charset_base = charset_base + "ČčĆćĐđŠšŽž"

print("source:", source_path)
print("output", output_path)
print("target", target_path)
print("charset:", charset_base)"""

'import datetime\nimport string\n\n# define parameters\nsource = "labels_w"\narch = "flor"\nepochs = 1000\nbatch_size = 16\n\n# define paths\nsource_path = os.path.join("..", "data", f"{source}.hdf5")\noutput_path = os.path.join("..", "output", source, arch)\ntarget_path = os.path.join(output_path, "checkpoint_weights.weights.h5")\nos.makedirs(output_path, exist_ok=True)\n\n# define input size, number max of chars per line and list of valid chars\ninput_size = (1024, 128, 1)\nmax_text_length = 128\ncharset_base = string.printable[:95]\ncharset_base = charset_base + "ČčĆćĐđŠšŽž"\n\nprint("source:", source_path)\nprint("output", output_path)\nprint("target", target_path)\nprint("charset:", charset_base)'

In [12]:
import datetime
import string

# define parameters
source = "mine_logs_v1"
arch = "flor"
epochs = 100  #1000
batch_size = 64   #16

# define paths
source_path = "../data/" + f"{source}.hdf5"
output_path = "../output/" + f"{source}/" + f"{arch}/"
target_path = output_path + "checkpoint_weights.weights.h5"
os.makedirs(output_path, exist_ok=True)

# define input size, number max of chars per line and list of valid chars
input_size = (1024, 128, 1)
max_text_length = 128
charset_base = string.printable[:95]
charset_base = charset_base + "ČčĆćĐđŠšŽž"

print("source:", source_path)
print("output", output_path)
print("target", target_path)
print("charset:", charset_base)

source: ../data/mine_logs_v1.hdf5
output ../output/mine_logs_v1/flor/
target ../output/mine_logs_v1/flor/checkpoint_weights.weights.h5
charset: 0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~ ČčĆćĐđŠšŽž


### 3.2 DataGenerator Class

In [13]:
import sys
sys.path.append('/mnt/c/AB_data/src')

In [14]:
from data.generator import DataGenerator

dtgen = DataGenerator(source=source_path,
                      batch_size=batch_size,
                      charset=charset_base,
                      max_text_length=max_text_length)

print(f"Train images: {dtgen.size['train']}")
print(f"Validation images: {dtgen.size['valid']}")
print(f"Test images: {dtgen.size['test']}")

Train images: 765
Validation images: 219
Test images: 110


### 3.3 HTRModel Class

In [15]:
print(tf.__version__)

2.19.0


In [16]:
from network.model import HTRModel

# create and compile HTRModel
model = HTRModel(architecture=arch,
                 input_size=input_size,
                 vocab_size=dtgen.tokenizer.vocab_size,
                 beam_width=10,
                 stop_tolerance=20,
                 reduce_tolerance=15,
                 reduce_factor=0.1)

model.compile(learning_rate=0.001)
model.summary(output_path, "summary.txt")

# get default callbacks and load checkpoint weights file (HDF5) if exists
model.load_checkpoint(target=target_path)

callbacks = model.get_callbacks(logdir=output_path, checkpoint=target_path, verbose=1)

I0000 00:00:1750077956.443529   17001 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9711 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:05:00.0, compute capability: 8.6


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 1024, 128, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 512, 64, 16)    │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu (PReLU)                 │ (None, 512, 64, 16)    │            16 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512, 64, 16)    │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ full_gated_conv2d               │ (None, 512, 64, 16)    │         4,640 │
│ (FullGatedConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 512, 64, 32)    │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_1 (PReLU)               │ (None, 512, 64, 32)    │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 512, 64, 32)    │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ full_gated_conv2d_1             │ (None, 512, 64, 32)    │        18,496 │
│ (FullGatedConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 256, 16, 40)    │        10,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_2 (PReLU)               │ (None, 256, 16, 40)    │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 256, 16, 40)    │           160 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ full_gated_conv2d_2             │ (None, 256, 16, 40)    │        28,880 │
│ (FullGatedConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256, 16, 40)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 256, 16, 48)    │        17,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_3 (PReLU)               │ (None, 256, 16, 48)    │            48 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 256, 16, 48)    │           192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ full_gated_conv2d_3             │ (None, 256, 16, 48)    │        41,568 │
│ (FullGatedConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256, 16, 48)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 256, 4, 56)     │        21,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_4 (PReLU)               │ (None, 256, 4, 56)     │            5

 Total params: 825,852 (3.15 MB)

 Trainable params: 825,340 (3.15 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input (InputLayer)              │ (None, 1024, 128, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 512, 64, 16)    │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu (PReLU)                 │ (None, 512, 64, 16)    │            16 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 512, 64, 16)    │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ full_gated_conv2d               │ (None, 512, 64, 16)    │         4,640 │
│ (FullGatedConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 512, 64, 32)    │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_1 (PReLU)               │ (None, 512, 64, 32)    │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 512, 64, 32)    │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ full_gated_conv2d_1             │ (None, 512, 64, 32)    │        18,496 │
│ (FullGatedConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 256, 16, 40)    │        10,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_2 (PReLU)               │ (None, 256, 16, 40)    │            40 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 256, 16, 40)    │           160 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ full_gated_conv2d_2             │ (None, 256, 16, 40)    │        28,880 │
│ (FullGatedConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256, 16, 40)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 256, 16, 48)    │        17,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_3 (PReLU)               │ (None, 256, 16, 48)    │            48 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 256, 16, 48)    │           192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ full_gated_conv2d_3             │ (None, 256, 16, 48)    │        41,568 │
│ (FullGatedConv2D)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256, 16, 48)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 256, 4, 56)     │        21,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ p_re_lu_4 (PReLU)               │ (None, 256, 4, 56)     │            5

 Total params: 825,852 (3.15 MB)

 Trainable params: 825,340 (3.15 MB)

 Non-trainable params: 512 (2.00 KB)

/home/haris/miniconda3/envs/TF-Py/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adamw', because it has 2 variables whereas the saved optimizer has 114 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


## 4 Training

In [17]:
# to calculate total and average time per epoch
start_time = datetime.datetime.now()

h = model.fit(x=dtgen.next_train_batch(),
              epochs=epochs,
              steps_per_epoch=dtgen.steps['train'],
              validation_data=dtgen.next_valid_batch(),
              validation_steps=dtgen.steps['valid'],
              callbacks=callbacks,
              shuffle=True,
              verbose=1)

total_time = datetime.datetime.now() - start_time

loss = h.history['loss']
val_loss = h.history['val_loss']

min_val_loss = min(val_loss)
min_val_loss_i = val_loss.index(min_val_loss)

time_epoch = (total_time / len(loss))
total_item = (dtgen.size['train'] + dtgen.size['valid'])

t_corpus = "\n".join([
    f"Total train images:      {dtgen.size['train']}",
    f"Total validation images: {dtgen.size['valid']}",
    f"Batch:                   {dtgen.batch_size}\n",
    f"Total time:              {total_time}",
    f"Time per epoch:          {time_epoch}",
    f"Time per item:           {time_epoch / total_item}\n",
    f"Total epochs:            {len(loss)}",
    f"Best epoch               {min_val_loss_i + 1}\n",
    f"Training loss:           {loss[min_val_loss_i]:.8f}",
    f"Validation loss:         {min_val_loss:.8f}"
])

with open(os.path.join(output_path, "train.txt"), "w") as lg:
    lg.write(t_corpus)
    print(t_corpus)

Epoch 1/100


E0000 00:00:1750077965.379846   17001 meta_optimizer.cc:967] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/functional_1/dropout_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
I0000 00:00:1750077966.368800   17075 cuda_dnn.cc:529] Loaded cuDNN version 90300


12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - loss: 61.8863   
Epoch 1: val_loss improved from inf to 62.77158, saving model to ../output/mine_logs_v1/flor/checkpoint_weights.weights.h5
12/12 ━━━━━━━━━━━━━━━━━━━━ 65s 3s/step - loss: 61.8729 - val_loss: 62.7716 - learning_rate: 0.0010
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 528ms/step - loss: 60.4645
Epoch 2: val_loss improved from 62.77158 to 60.25947, saving model to ../output/mine_logs_v1/flor/checkpoint_weights.weights.h5
12/12 ━━━━━━━━━━━━━━━━━━━━ 8s 634ms/step - loss: 60.4771 - val_loss: 60.2595 - learning_rate: 0.0010
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 531ms/step - loss: 60.7344
Epoch 3: val_loss improved from 60.25947 to 56.18571, saving model to ../output/mine_logs_v1/flor/checkpoint_weights.weights.h5
12/12 ━━━━━━━━━━━━━━━━━━━━ 8s 638ms/step - loss: 60.6193 - val_loss: 56.1857 - learning_rate: 0.0010
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 546ms/step - loss: 58.9740
Epoch 4: val_loss did not improve from 56.18571
12/12 ━━